Start Project: look at data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns


# vocab = torchtext.vocab.build_vocab_from_iterator(train_X["comment_text"],min_freq=5,specials=["<unk>","<pd>","<eos>"],special_first=True)
# vocab.set_default_index(vocab["<unk>"])

## Create Dataset Class ##
We will use this Dataset for training,and testing. we made it very general but usually we will do more preprossecing in the __init__ function. 

In [21]:
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset
class TransformDataset(Dataset):
    def __init__(self,file,tokenizer, transform = None,target_transform = None):
        # self.transform = transform
        # self.target_transform = target_transform
        data = pd.read_csv(file)
        X = data["comment_text"].apply(lambda row: tokenizer(row) + ["<eos>"]).values
        d = data.values
        y = None
        if d.shape[1] > 2:
            y = data.values[:,2:].astype(np.int32)
        self.X = X
        self.y = y
        self.vocab = None
    
    def get_vocab(self):
        if self.vocab is None:
            self.vocab = build_vocab_from_iterator(self.X,min_freq=5,specials=["<unk>","<pd>","<eos>"],special_first=True)
            self.vocab.set_default_index(self.vocab["<unk>"])
        return self.vocab
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        item = self.X[idx]
        label = self.y[idx] if self.y is not None else None
        # if self.transform:
        #     item = self.transform(item)
        # if self.target_transform:
        #     label = self.target_transform(label)
        return item,label
        
        

In [22]:
# from torchtext.data.utils import get_tokenizer
# from torchtext.vocab import build_vocab_from_iterator
# def convert_data(file,return_vocab = True):
#     data = pd.read_csv(file)
#     tokenizer = get_tokenizer("basic_english")
#     X = data["comment_text"].apply(lambda row: tokenizer(row) + ["<eos>"]).values
#     d = data.values
#     y = None
#     if d.shape[1] > 2:
#         y = data.values[:,2:]
#     if return_vocab:
#         vocab = torchtext.vocab.build_vocab_from_iterator(X,min_freq=5,specials=["<unk>","<pd>","<eos>"],special_first=True)
#         vocab.set_default_index(vocab["<unk>"])
#         return X,y,vocab
#     return X,y
    

In [18]:
# X,y,vocab = convert_data("train.csv")

In [19]:
from torchtext.transforms import VocabTransform,ToTensor
from torchvision.transforms import Compose
from torchtext.data.utils import get_tokenizer
import torch

device = "cuda" if torch.cuda.is_available else "cpu"

In [25]:
tokenizer = get_tokenizer("basic_english")
train = TransformDataset("train.csv",tokenizer)
vocab = train.get_vocab()
transform = Compose([VocabTransform(vocab),ToTensor(vocab(["<pd>"])[0],dtype=torch.int32)])


In [33]:
def collate_batch(batch):
    print(batch[0][1])
    X = [item[0] for item in batch]
    y = np.array([item[1] for item in batch])
    X = transform(X)
    return X.to(device),torch.tensor(y,dtype=torch.int32,device=device)

In [34]:
from torch.utils.data import DataLoader


In [35]:
dataloader = DataLoader(train,batch_size=4,shuffle=True,collate_fn=collate_batch)

for X,y in dataloader:
    print(X)
    print(y)
    break

[0 0 0 0 0 0]
tensor([[   61,  4722,  3863,    83,   114,    12,  2982,    63,    19,   453,
            90,    16,    12,  1037,    34,    28,    23,    40, 13438,    21,
             4,   645,   292,    14,     4,  2256,     8,     4,   723, 16037,
            14,     8,  5867,  2716,     5,   120,    25,     0,     5,     0,
             5,    36,     0,     0, 14982,  4602,     5,     9,    14,   102,
           723,  4645,     0,  2256,     5,     0,     0,     5,     9,     0,
             0,     3,  9844,     0,    54,  2635,    20,     6,   951,   851,
             8,    83,     4,   723,   755,    36,   790,     8,   119,  2256,
            49,  1180,    41, 11132,   384,    54,    68,   724,     6,  1133,
            16,    12,  1037,    34,    66,    12,   384,    14,   404,   541,
          1294,    18,     4,   645,     3,    14,    19,    54,    68,  3321,
             6,  1037,    34,   663,   772,     3,   100,   301,   645,    55,
           106,   102,  3998,    84,  

we have the training data. now we would like to get some embedding for the words. I will try to use self attention to create context rich embeddings